In [3]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from astropy.cosmology import default_cosmology
from astropy.units import Quantity
from slsim.Pipelines.skypy_pipeline import SkyPyPipeline
from slsim.Pipelines.sl_hammocks_pipeline import SLHammocksPipeline
from slsim.lens_pop import LensPop
import slsim.Deflectors as deflectors
import slsim.Sources as sources
import corner
from pprint import pprint
    
from mejiro.exposure import Exposure
from mejiro.synthetic_image import SyntheticImage
from mejiro.instruments.roman import Roman

%matplotlib inline

In [2]:
# Import default Roman Space Telescope configuration
skypy_config = "roman-like.yml"

# Load Roman filters
speclite_filters = Roman.load_speclite_filters(sca=1)

In [4]:
# define a cosmology
cosmo = default_cosmology.get()

# define a sky area
sky_area = Quantity(value=1., unit="deg2")

# run skypy pipeline and make galaxy-galaxy population class
galaxy_simulation_pipeline = SkyPyPipeline(
    skypy_config=skypy_config,
    sky_area=sky_area,
    filters=speclite_filters,
    cosmo=cosmo
)

In [ ]:
# Generate halo galaxy population using SLHammock pipeline. Halo galaxies from this simulation
# will be used as deflector population. One can specify minimum and maximum values for
# redshift and log10 of halo mass.
halo_galaxy_pipeline = SLHammocksPipeline(
    slhammocks_config=None,
    sky_area=sky_area,
    # filters=speclite_filters,
    cosmo=cosmo,
    # z_min=0.01,
    # z_max=2,
    # loghm_min=13,
    # loghm_max=16,
)

In [12]:
halo_galaxy_pipeline._pipeline

z,halo_mass,halo_mass_acc,e_h,p_h,concentration,stellar_mass,ellipticity,p_g,tb,angular_size,vel_disp,mag_g,mag_r,mag_i,mag_z,mag_Y,e1_light,e2_light,e1_mass,e2_mass
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
0.01999999999999999,694791602867.4675,0.0,0.7067663632235568,-135.70639232536487,8.610360360483075,35101320731.23165,0.6315315439699243,-93.79623531165643,5.930033202633122,5.217715473811859e-05,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
0.02099999999999999,436607404475.2233,0.0,0.05801650119904066,169.28464321850757,9.985437840144938,12012743482.486063,0.5124911428085256,154.5589458734267,10.035695128066061,8.830203806770858e-05,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
0.03299999999999998,1444466374097.4253,0.0,0.2554613221796741,79.4042546391634,4.41999985921903,85358766175.61157,0.2842724185496868,97.7789201742155,5.286914068333569,4.651848042066407e-05,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
0.035999999999999976,353423023489.2082,0.0,0.4000234160142594,157.6559204730321,8.982299307405654,8667622907.640635,0.12777727999530003,136.5792607816346,9.392742116542529,8.264482543073607e-05,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
0.03899999999999998,291785155426.9099,0.0,0.20625510126917088,-129.90591897056248,5.912055939426572,9155274756.372576,0.368180674616726,-154.35334610667007,3.801258469628636,3.344649930144211e-05,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
0.04399999999999998,217448736311.4261,0.0,0.43019704175511053,75.41347650767491,4.4714289965304,3355124880.952329,0.3551434587165091,86.20635168407523,2.2698528213216402,1.997197281092214e-05,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
0.04499999999999997,7107771711723.727,0.0,0.29138116913046136,176.08799081974095,6.274899559066285,139545697735.39462,0.7375238005940139,107.53159061427294,1.2411147869035544,1.0920316307045181e-05,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
0.04599999999999997,206271237142.06345,0.0,0.266527229233353,-107.70998559267785,9.11195520053984,2912965900.206084,0.3445639050481749,-98.873250877848,1.5501765608082467,1.3639687927679973e-05,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
0.04599999999999997,223027512569.04236,0.0,0.2291915928466484,-96.23920305502014,9.085485523642193,2840957417.191611,0.36612215542850485,-92.9023619877346,1.4932513953624147,1.3138814987433817e-05,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [11]:
# define limits in the intrinsic deflector and source population (in addition to the skypy config
# file)
kwargs_deflector_cut = {"band": "F129", "band_max": 28, "z_min": 0.01, "z_max": 2.}
kwargs_source_cut = {"band": "F129", "band_max": 28, "z_min": 0.1, "z_max": 5.}

# Initiate deflector population class.
lens_galaxies = deflectors.CompoundLensHalosGalaxies(
    halo_galaxy_list=halo_galaxy_pipeline._pipeline,
    kwargs_cut=kwargs_deflector_cut,
    kwargs_mass2light={},
    cosmo=cosmo,
    sky_area=sky_area,
)

# Initiate source population class.
source_galaxies = sources.Galaxies(
    galaxy_list=galaxy_simulation_pipeline.blue_galaxies,
    kwargs_cut=kwargs_source_cut,
    cosmo=cosmo,
    sky_area=sky_area,
    catalog_type="skypy",
)

# make galaxy-galaxy population class using LensPop
gg_lens_pop = LensPop(
    deflector_population=lens_galaxies,
    source_population=source_galaxies,
    cosmo=cosmo,
    sky_area=sky_area,
)

KeyError: 'mag_F129'

In [ ]:
kwargs_lens_cut = {"min_image_separation": 0.8, "max_image_separation": 10}
lens_population = gg_lens_pop.draw_population(kwargs_lens_cuts=kwargs_lens_cut)

In [ ]:
print("Number of lenses:", len(lens_population))

lens_samples = []
labels = [
    r"$\sigma_v$",
    r"$\log(M_{*})$",
    r"$\log(h_{m})$",
    r"$\theta_E$",
    r"$z_{\rm l}$",
    r"$z_{\rm s}$",
    r"$m_{\rm source}$",
    r"$m_{\rm lens}$",
]

for gg_lens in lens_population:
    vel_disp = gg_lens.deflector_velocity_dispersion()
    m_star = gg_lens.deflector_stellar_mass()
    halo_mass = gg_lens.deflector.halo_properties[0]
    theta_e = gg_lens.einstein_radius[0]
    zl = gg_lens.deflector_redshift
    zs = gg_lens.source_redshift_list[0]
    source_mag = gg_lens.extended_source_magnitude(band="i", lensed=True)[0]
    deflector_mag = gg_lens.deflector_magnitude(band="i")
    lens_samples.append(
        [
            vel_disp,
            np.log10(m_star),
            np.log10(halo_mass),
            theta_e,
            zl,
            zs,
            source_mag,
            deflector_mag,
        ]
    )

hist2dkwargs = {
    "plot_density": False,
    "plot_contours": False,
    "plot_datapoints": True,
    "color": "b",
    "data_kwargs": {"ms": 5},
}
corner.corner(np.array(lens_samples), labels=labels, **hist2dkwargs)
plt.show()